In [1]:
import pandas as pd
import numpy as np
import warnings
warnings. simplefilter(action='ignore', category=Warning)

In [2]:
def GoldenCrossverSignal(name):
    path = f'../2)data/{name}.csv'
    data = pd.read_csv(path, parse_dates=['Date'], index_col='Date')
    data['Prev_Close'] = data.Close.shift(1) #we use prev_close to remove forwad bias
    data['50_DMA'] = data.Prev_Close.rolling(window=50, min_periods=1).mean()
    data['200_DMA'] = data.Prev_Close.rolling(window=200, min_periods=1).mean()
    data['Signal'] = 0
    data['Signal'] = np.where(data['50_DMA'] > data['200_DMA'], 1, 0)
    data['Position'] = data.Signal.diff()
    df_pos = data[(data['Position'] == 1) | (data['Position'] == -1)].copy()
    df_pos['Position'] = df_pos['Position'].apply(lambda x: 'Buy' if x == 1 else 'Sell')
    return df_pos

In [3]:
data = GoldenCrossverSignal('RELIANCE')

In [4]:
req_data = data[(data.index >= data[data['Position'] == 'Buy'].index[0]) & (data.index <= data[data['Position'] == 'Sell'].index[-1])]

In [5]:
# Name, Entry TIme, Entry PRice, QTY, Exit Time, Exit Price
class Backtest:
    def __init__(self):
        self.columns = ['Equity Name', 'Trade', 'Entry Time', 'Entry Price', 'Exit Time', 'Exit Price', 'Quantity', 'Position Size', 'PNL', '% PNL']
        self.backtesting = pd.DataFrame(columns=self.columns)

    def buy(self, equity_name, entry_time, entry_price, qty):
        self.trade_log = dict(zip(self.columns, [None] * len(self.columns)))
        self.trade_log['Trade'] = 'Long Open'
        self.trade_log['Quantity'] = qty
        self.trade_log['Position Size'] = round(self.trade_log['Quantity'] * entry_price, 3)
        self.trade_log['Equity Name'] = equity_name
        self.trade_log['Entry Time'] = entry_time
        self.trade_log['Entry Price'] = round(entry_price, 2)

    def sell(self, exit_time, exit_price, exit_type, charge):
        self.trade_log['Trade'] = 'Long Closed'
        self.trade_log['Exit Time'] = exit_time
        self.trade_log['Exit Price'] = round(exit_price, 2)
        self.trade_log['Exit Type'] = exit_type
        self.trade_log['PNL'] = round((self.trade_log['Exit Price'] - self.trade_log['Entry Price']) * self.trade_log['Quantity'] - charge, 3)
        self.trade_log['% PNL'] = round((self.trade_log['PNL'] / self.trade_log['Position Size']) * 100, 3)
        self.trade_log['Holding Period'] = exit_time - self.trade_log['Entry Time']
        new = pd.DataFrame([self.trade_log])
        self.backtesting = pd.concat([new,self.backtesting], ignore_index=True)

In [6]:
bt = Backtest()
capital = 10000
for index, data in req_data.iterrows():
    if(data.Position == 'Buy'):
        qty = capital // data.Open
        bt.buy('RELIANCE', index, data.Open, qty)
    else:
        bt.sell(index, data.Open, 'Exit Trigger', 0)

In [7]:
bt.backtesting.to_csv('RELIANCE_TEST.csv')
bt.backtesting.PNL.sum()

92655.0